<img src="https://github.com/rmlarose/qcbq/blob/master/img/banner.png?raw=true" alt="QCBQ Banner">

# Implementing Variational Quantum Algorithms

**Author:** Ryan LaRose

(lightly modified by Justin for use in the QuIC seminar)

In this notebook, we'll introduce a class of circuits/algorithms known as **variational quantum algorithms**. Variational algorithms are a class of algorithms that may foreseeably be implemented on near term intermediate scale quantum computers, as opposed to things like Shor's factoring algorithm which require large scale, fault tolerant quantum computers. The idea is to build circuits with parameters we may vary, estimate the expectation values of those circuits, compute some cost function, and implement classical optimization algorithms so we can figure out how to vary our circuit to get closer to the true answer. Qiskit has a package (Qiskit Aqua) that does this all behind the scenes, but it's important to understand what's actually going on in the code, so we'll implement all these steps ourselves!

## Learning goals

(1) Be able to implement a circuit with variable parameters and update these parameters.

(2) Be able to estimate expectation values using a quantum circuit.

(3) Understand how to compute a cost function with a quantum circuit.

(4) Use a classical optimization algorithm to find the best parameters.

## Helpful background

* [Born's rule](https://en.wikipedia.org/wiki/Born_rule) for probabilities in quantum mechanics.

In [ ]:
"""Imports for the notebook."""
import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import minimize

import qiskit as qis

## Circuits with parameters

A key characteristic of any variational quantum algorithm is a parameterized circuit. More generally, variational quantum algorithms are instances of the well-known variational principle of quantum mechanics. This principle states that you can never get below the ground state energy of a particular Hamiltonian. The proof of this theorem is "duh, ground state means lowest energy." The formal proof is just a mathematical translation of this fact.

**Theorem (The Variational Principle):** Let $H = H^\dagger$ be a Hamiltonian with spectrum (energy) $E_0 \le E_1 \le \cdots \le E_n$. Then, for any valid wavefunction $|\psi\rangle$,

$$ E_0 \le \langle \psi | H | \psi \rangle .$$

*Proof*: Write $|\psi\rangle$ in the eigenbasis of $H$

$$ |\psi\rangle = \sum_n c_n |\psi_n\rangle $$

where $c_n := \langle \psi | \psi_n \rangle$ and $H |\psi_n \rangle = E_n |\psi_n\rangle$. Note that $\sum_n |c_n|^2 = 1$ by virtue of proper normalization. Now,

\begin{align}
    \langle \psi | H | \psi \rangle &= \left[ \sum_m c_m | \psi_m \rangle \right]^\dagger H \left[ \sum_n c_n |\psi_n\rangle \right] \\
    &= \sum_{m, n} E_n c_m^* c_n \langle \psi_m | \psi_n \rangle \\
    &= \sum_n E_n |c_n|^2 \\
    &\ge E_0 \sum_n |c_n|^2 \\
    &= E_0 .
\end{align}

_______________________________________________________________________________

The **big idea** in all variational methods is to prepare some "**ansatz"** wavefunction

$$ |\psi\rangle = |\psi(\mathbf{\alpha}) \rangle $$

parameterized by $\mathbf{\alpha} = (\alpha_1, \alpha_2, ..., \alpha_n)$. When solving quantum mechanics problems, we then vary $\mathbf{\alpha}$ to try and minimize $\langle\psi(\alpha)|\mathbf{H}|\psi(\alpha)\rangle$ and find a wavefunction close to the ground state wavefunction. In the context of quantum computing, we prepare states by implementing gates, so our gates should have some tunable parameters in them. This is what we mean by "circuits with parameters."

### Defining circuits with parameters

The code cell below shows an example of defining a two qubit circuit with two parameters. Run this cell to visualize the circuit.

In [ ]:
"""Defining circuits with parameters."""
# Get a circuit and registers
qreg = qis.QuantumRegister(2)
creg = qis.ClassicalRegister(2)
circ = qis.QuantumCircuit(qreg, creg)

# Add gates with particular parameters
circ.h(qreg)
circ.rx(0.2, qreg[0])
circ.cx(qreg[0], qreg[1])
circ.ry(0.1, qreg[1])

# Visualize the circuit
print(circ)

The angles come in through single qubit rotation gates. For this example, we instantiated these angles with example (arbitrary) values. Generally, it is useful to have some means of quickly generating the same circuit but with different variational parameters. In terms of programming, this can be done in a variety of ways, including symbolic representation of parameters or functions. We'll use the latter case for this notebook.

<font size=8 color="#009600">&#9998;</font> **Do this:** In the cell below, write a function in python which inputs two paramters (floats) and returns the above circuit with rotation angles equal to these parameters. If you've never written a python function before, ask for help! It's super easy (no curly brackets required). Once you've done that, verify you can call your function by running the cell under that.

In [ ]:
def circuit(alpha1: float, alpha2: float):
    """Returns the circuit above with the input parameters."""
    ### Your code here!
    
    return circ

In [ ]:
my_circ = circuit(0.2,0.3)
print(my_circ)

We can now easily vary the parameters in our circuit! You could of course do this by creating a new cell and calling the function with several different parameters. (You should do this to at least make sure your function works!) 

The reason we vary parameters is to minimize some energy (which comes from a cost function/Hamiltonian). This involves two things:

1. Computing the energy.
1. Varying the parameters.

We'll break down computing energy into two sub-steps: Individual expectation values, and weighted sums of expectation values. 

## Computing expectation values

So far, we know that variational algorithms tackle the problem

$$ \min_{\alpha} \langle \psi(\alpha) | H | \psi(\alpha) \rangle $$

**How do we compute expectation values like $\langle \psi | H | \psi \rangle$ on a quantum computer?**

### A diagonal operator

Suppose first we have a one qubit wavefunction $|\psi\rangle$ and we want to compute 

$$\langle \psi | Z | \psi \rangle.$$

where $Z$ is the usual Pauli suspect

$$ Z = \left[ \begin{matrix}
    1 & 0 \\
    0 & -1 \\
\end{matrix} \right] $$

in the computational basis.

Starting from first principles, Born's rule tells us that the probability of measuring $0$ is

$$ p(0) = | \langle \psi | 0 \rangle |^2 = 
\langle \psi | 0 \rangle \langle 0 | \psi \rangle =
\langle \psi | \Pi_0 | \psi \rangle $$

where $\Pi_0 := |0\rangle \langle 0 |$ is the projector onto the $|0\rangle$ state. 

**Question:** Write out the $2 \times 2$ matrix representation of $\Pi_0$ in the computational basis.

<font size=8 color="#009600">&#9998;</font> **Answer:** Answer the above question here!

Similarly, the probability of measuring $1$ is

$$ p(1) = | \langle \psi | 1 \rangle |^2 = 
\langle \psi | 1 \rangle \langle 1 | \psi \rangle =
\langle \psi | \Pi_1 | \psi \rangle $$

where $\Pi_1 := |1 \rangle \langle 1 |$ is the projector onto the $|1\rangle$ state. 

**Question:** Write out the $2 \times 2$ matrix representation of $\Pi_1$ in the computational basis.

<font size=8 color="#009600">&#9998;</font> **Answer:** Answer the above question here!

**Question:** Prove that $\Pi_0 + \Pi_1 = I$ (the identity). Use this to check that $p(0) + p(1) = 1$, as it must.

<font size=8 color="#009600">&#9998;</font> **Answer:** Answer the above question here!

The previous question showed that if we add together the projectors $\Pi_0$ and $\Pi_1$, nothing interesting happens. The follow question shows that if we **subtract** the projectors, something interesting happens!

**Question:** Prove that $\Pi_0 - \Pi_1 = Z$. 

<font size=8 color="#009600">&#9998;</font> **Answer:** Answer the above question here!

As you showed above, we have

$$ Z = \Pi_0 - \Pi_1. $$

We can use this to estimate $\langle \psi | Z | \psi \rangle$ by measuring in the standard basis and doing a bit of "classical postprocessing," which here just means subtracting the outcome probabilities.

$$ \langle \psi | Z | \psi \rangle = \langle \psi | (\Pi_0 - \Pi_1) | \psi \rangle =
\langle \psi | \Pi_0 | \psi \rangle - \langle \psi | \Pi_1 | \psi \rangle = 
p(0) - p(1) . $$

Of course, we won't have the full probability distribution $p$, but instead we'll have to sample from the circuit many times ($N$ times) to get a good estimate $p(0) \approx f(0) / N$, where $f(0)$ is the frequency of measuring $0$. Similarly for $p(1)$.

<font size=8 color="#009600">&#9998;</font> **Do this:** In the following cell, a one qubit state $|\psi\rangle = H|0\rangle$ is prepared for you in a circuit. 

1. Estimate $\langle \psi | Z | \psi \rangle$ by executing the circuit many times and doing the appropriate "classical post-processing."
1. Compute the expectation analytically, and show the results agree.

In [ ]:
"""Estimating a one qubit expectation value."""
qreg = qis.QuantumRegister(1)
creg = qis.ClassicalRegister(1)
circ = qis.QuantumCircuit(qreg, creg)
circ.h(qreg)

### Your code here!



### A non-diagonal operator

Suppose now, for example, we want to measure the expecation value of $X$, which is not diagonal in the computational basis:

$$\langle \psi | X | \psi \rangle.$$

Here, the key "trick" is to rotate to the eigenbasis, where $X$ becomes diagonal. You may recall or wish to prove that

$$ HXH = Z . $$

Let $|\psi'\rangle = H |\psi \rangle$, and suppose what happens when we measure $|\psi'\rangle$ in the computational basis. By the same argument above, we have

$$ p(0) = \langle \psi ' | \Pi_0 | \psi' \rangle = 
\langle \psi | H \Pi_0 H | \psi \rangle,$$

where in the last step we substituted $|\psi'\rangle = H |\psi\rangle$. Similarly,

$$ p(1) = \langle \psi ' | \Pi_1 | \psi' \rangle = 
\langle \psi | H \Pi_1 H | \psi \rangle . $$

We can now subtract the probabilities, again using the fact that $Z = \Pi_0 - \Pi_1$, as above:

$$ p(0) - p(1) = \langle \psi | H \Pi_0 H | \psi \rangle - \langle \psi | H \Pi_1 H | \psi \rangle = 
\langle \psi | H (\Pi_0 - \Pi_1 ) H | \psi \rangle = 
\langle \psi | H Z H | \psi \rangle =
\langle \psi | X | \psi \rangle, $$

which is exactly the quantity we want to compute. 

This is why we implement the "appropriate rotation" when we want to measure the expectation value of an operator. The key insight is that measurement probabilities can be written as expectation values of projectors.

<font size=8 color="#009600">&#9998;</font> **Do this:** In the following cell, a one qubit state $|\psi\rangle = |-\rangle = 1/\sqrt{2}(|0\rangle - |1\rangle)$ is prepared for you in a circuit. 

1. Estimate $\langle \psi | X | \psi \rangle$ by executing the circuit many times and doing the appropriate "classical post-processing."
1. Compute the expectation analytically, and show the results agree.

Note that the output of counts is a [dictionary](https://docs.python.org/3/library/stdtypes.html#typesmapping) containing the result paired to the number of counts. If your circuit produces no counts of a certain result, it won't output a dictionary element for that result, so you may have to include an ```if``` statement to account for that.

In [ ]:
"""Estimating a one qubit expectation value."""
qreg = qis.QuantumRegister(1)
creg = qis.ClassicalRegister(1)
circ = qis.QuantumCircuit(qreg, creg)
circ.x(qreg)
circ.h(qreg)

### Your code here!


### A two-qubit operator

We now know how to estimate $\langle Z \rangle$ and $\langle X \rangle$. **What if we want to estimate $\langle Z \otimes X \rangle$?**

The trick is to do the same thing! We still rotate to the eigenbasis of each operator and measure both qubits. We now have four possible measurement outcomes and so four probabilities: $p(00)$, $p(01)$, $p(10)$, and $p(11)$. What is the appropriate "classical post-processing" to do with these sampled outcomes?

We can measuring the expectations separately and expand the product:

$$ (p_0(0) - p_0(1))(p_1(0) - p_1(1)) = p(00) - p(01) - p(10) + p(11). $$

Note that the subscript on the LHS refers to the qubit (either $0$ or $1$), and the value in parentheses refers to the measurement outcome. On the RHS, $p(00)$ is equivalent to $p_0(0) p_1(0)$, etc. 

<font size=8 color="#009600">&#9998;</font> **Do this:** In the following cell, a two qubit state $|\psi\rangle = |+\rangle = 1/\sqrt{2}(|0\rangle + |1\rangle)$ is prepared for you in a circuit. 

1. Estimate $\langle \psi | Z \otimes X | \psi \rangle$ by executing the circuit many times and doing the appropriate "classical post-processing."
1. Compute the expectation analytically, and show the results agree.

In [ ]:
"""Estimating a two qubit expectation value."""
qreg = qis.QuantumRegister(2)
creg = qis.ClassicalRegister(2)
circ = qis.QuantumCircuit(qreg, creg)
circ.h(qreg[1])

### Your code here!

### A general operator

Thankfully for our discussion, any $n$-qubit operator $O$ can be decomposed in the Pauli basis:

$$ O = \sum_i o_i \sigma_{i_1} \otimes \sigma_{i_2} \otimes \cdots \otimes \sigma_{i_n} $$

Here, $o_i \in \mathbb{C}$ is a scalar and each $\sigma$ is a Pauli. By linearity, to evaluate the expectation, we can evaluate the expectation of each Pauli string:

$$ \langle O \rangle = \sum_i o_i \langle \sigma_{i_1} \otimes \sigma_{i_2} \otimes \cdots \otimes \sigma_{i_n} \rangle $$

Thus, **to evaluate the expectation of any operator, it suffices to evaluate expectations of arbitrary Pauli strings**. Below, you are asked to write a function which generalizes the two qubit expectation value measurement $\langle Z \otimes X \rangle$ to any number of qubits. Before tackling this, you may wish to prove the following fact to yourself, based on the expansion of the two-qubit expectation above.

**Question**: We can think of the classical post-processing for computing expectation values as follows. From our circuit of $n$ qubits, we measure the frequency of $2^n$ possible bitstrings. For example, for $n = 3$, the possible bitstrings we can measure are 000, 001, 010, 011, 100, 101, 110, and 111. The classical post-processing consists of summing up these frequencies **with the appropriate sign**. Prove that the sign for bitstring $z_i$ is $(-1)^{N_0(z_i)}$ where $N_0(z_i)$ is the number of zeros in bit string $z_i$. 

<font size=8 color="#009600">&#9998;</font> **Do this:** Answer the above question here!

The block of code below contains a function ```expectation_circuit``` for finding appropriately modifying a circuit to measure the expectation value of a general string of operators. The function takes in an $n$ qubit circuit and a string of length $n$ containing pauli operators, and outputs the appropriate circuit to run to estimate the expectation value of the operator defined by the string. For example, if we had a 3 qubit circuit and wanted to measure $I\otimes X \otimes Z$, we would pass the circuit and ``` "IXZ" ``` to the function. 

Read through this function, and make sure you understand what's going on!

In [ ]:
"""Helper function to evaluate the expectation of any valid Pauli string."""
def expectation_circuit(circuit: qis.QuantumCircuit, pauli_string: str) -> qis.QuantumCircuit:
    """Returns a circuit to compute expectation of the Pauli string in the 
    state prepared by the input circuit.
    
    Args:
        circuit: Prepares the state |\psi> from |0>.
        pauli_string: String (tensor product) of Paulis to evaluate
                      an expectation of. The length of pauli_string
                      must be equal to the total number of qubits in
                      the circuit. (Use identities for no operator!)
    """
    temp = circuit.copy()
    
    if len(circuit.qregs) != 1:
        raise ValueError("Circuit should have only one quantum register.")
    if len(circuit.cregs) != 1:
        print("# cregs =", len(circuit.cregs))
        raise ValueError("Circuit should have only one classical register.")
    
    qreg = circuit.qregs[0]
    creg = circuit.cregs[0]
    nqubits = len(qreg)
    pauli_string = pauli_string.upper().strip()
    
    if len(pauli_string) != nqubits:
        raise ValueError(
            f"Circuit has {nqubits} qubits but pauli_string has {len(pauli_string)} operators."
        )
    
    for (qubit, pauli) in enumerate(pauli_string):
        if pauli == "I":
            continue
        elif pauli == "X":
            temp.h(qreg[qubit])
            temp.measure(qreg[qubit], creg[qubit])
        elif pauli == "Y":
            temp.s(qreg[qubit])
            temp.h(qreg[qubit])
            temp.measure(qreg[qubit], creg[qubit])
        elif pauli == "Z":
            temp.measure(qreg[qubit], creg[qubit])
        else:
            raise ValueError(f"{pauli} is an invalid Pauli string key. Should be I, X, Y, or Z.")
    
    return temp

<font size=8 color="#009600">&#9998;</font> **Do this:** As a sanity check, let's test this function below on measuring $X \otimes Y$. Use the function to modify the input circuit, then print out the modified circuit. Test this on other Pauli strings as well.

In [ ]:
"""Test your function here."""
circ= circuit(np.pi / 2, np.pi / 4)
print("Bare circuit:")
print(circ)

### Your code here!


The following cell contains a function that takes in an "expectation circuit" (a circuit with the appropriate transformations applied at the end) and performs the classical post-processing to return an expectation value. Once again, read through this function and make sure you understand what's going on!

In [ ]:
"""Function to execute the circuit and do the postprocessing."""
def run_and_process(circuit: qis.QuantumCircuit, shots: int = 10000) -> float:
    """Runs an 'expectation circuit' and returns the expectation value of the
    measured Pauli string.
    
    Args:
        circuit: Circuit to execute.
        shots: Number of circuit executions.
    """
    
    # Execute the circuit
    backend = qis.BasicAer.get_backend("qasm_simulator")
    job = qis.execute(circuit, backend, shots=shots)
    counts = job.result().get_counts()
    
    # Do the postprocessing
    val = 0.0
    for bitstring, count in counts.items():
        sign = (-1) ** bitstring.count("0")
        val += sign * count
    
    return val / shots

<font size=8 color="#009600">&#9998;</font> **Do this:** In the following cell, combine the previous two functions into one function which takes in a circuit and computes the expectation value of a Pauli string.

In [ ]:
"""Define your function here!"""
def expectation(circuit: qis.QuantumCircuit, pauli_string: str, shots: int = 10000) -> float:
    """Returns the expectation value of the pauli string in the state prepared by the circuit."""
    
    ### Your code here!
  

<font size=8 color="#009600">&#9998;</font> **Do this:** Now test your function on a variety of circuits below, and make sure it gives sensible (correct) results!

In [ ]:
"""
Test your function here
Add some gates to the circuit, and change around the Paulis
Try some choices with obvious answers and make sure they make sense

"""
qreg = qis.QuantumRegister(2)
creg = qis.ClassicalRegister(2)
circ = qis.QuantumCircuit(qreg, creg)

pauli = 'II'

### Your code here!


## Computing the cost function

Now that we can evaluate expectation values of arbitrary Pauli strings, we're in great shape. Recall that *any* operator can be decomposed into a sum of weighted Pauli operators. We can write down our Hamiltonian as a sum of Pauli operators, and simply calculate the expectation value of each operator independently and add them up!

<font size=8 color="#009600">&#9998;</font> **Do this:** In the following cell, use your function defined above to compute the expectation value of the operator

$$ H = 0.5 IZZ + -0.3 ZZI + 1.2 ZIZ $$ 

in the state given by the provided circuit.

In [ ]:
"""Compute the expectation of a Hamiltonian."""
# Provided circuit
qreg = qis.QuantumRegister(3)
creg = qis.ClassicalRegister(3)
circ = qis.QuantumCircuit(qreg, creg)
circ.h(qreg)
circ.rx(np.pi / 4, qreg[0])
circ.cz(qreg[0], qreg[1])
circ.cz(qreg[1], qreg[2])
print(circ)

weights = (0.5, -0.3, 1.2)
paulis = ("IZZ", "ZZI", "ZIZ")

### Your code here


For this notebook, we consider cost functions of a general form. The particular form of your Hamiltonian always comes from the problem of interest. For example, at the bootcamp we talked about solving the MaxCut problem using the Quantum Approximate Optimization Algorithm (QAOA), for which we use the MaxCut Hamiltonian: 

\begin{equation}
    H_C = \frac{1}{2} \sum_{(i, j) \in E} w_{ij} (1 - z_i z_j) .
\end{equation}

Other Hamiltonians can come from molecules, other combinatorial optimization problems, etc. For this notebook, we'll be satisfied to know how to compute expectation values *given a Hamiltonian*. Converting a Hamiltonian into a suitable form and deriving such a Hamiltonian will not be covered.

<font size=8 color="#009600">&#9998;</font> **Do this:** In the following cell, write a function which takes in a list of Pauli operators with coefficients (**weighted Pauli operators**) and returns the expectation value taken in an input circuit.

In [ ]:
"""Function to compute the cost of any Hamiltonian in the state prepared by the circuit."""
def cost(circuit, weights, paulis):
    """Returns <psi|H|psi> where |psi> is prepared by the circuit
    and the weights and paulis define a Hamiltonian H.
    
    Args:
        circuit: Circuit which prepares a state.
        weights: List of floats which are the coeffs/weights of each Pauli string.
        paulis:  List of strings which specify the Paulis.
    """
    if len(weights) != len(paulis):
        raise ValueError("Args weights and paulis must have the same length.")
    ### Your code here!
    

<font size=8 color="#009600">&#9998;</font> **Do this:** In the following cell, use your function  to compute the expectation value (cost) of the Hamiltonian given above in the circuit given above. Make sure your reults agree with your answer above. Also test your function on other cases.

In [ ]:
"""Evaluate your cost here!"""
### Your code here!



<font size=8 color="#009600">&#9998;</font> **Do this:** The following cell contains a function that outputs a one qubit circuit with a variable $X$ rotation, i.e. one-qubit parameterized state $|\psi(\alpha)\rangle$. Use your cost function finder to compute the cost (energy) of the Hamiltonian $H = Z$ as a function of $\alpha$ (the variable parameter) and plot the resulting 'cost landscape' using matplotlib. Interpret your result: does it make sense?

In [ ]:
"""Plot a cost landscape."""
def oneq_circ(param):
    qreg = qis.QuantumRegister(1)
    creg = qis.ClassicalRegister(1)
    circ = qis.QuantumCircuit(qreg, creg)
    circ.rx(param, qreg)
    return circ

weights = (1.0,)
paulis = ("Z",)

pvals = np.linspace(-np.pi, np.pi, 100)
cvals = []

### Your code here!


## Minimizing the cost function

We can now compute expectation values of Hamiltonians, which we want to minimize. Such a quantity to be minimized is known as a **cost** or **objective** function. There are very many methods to adjust parameters in hopes of finding a minimum cost. A few of these are grid search, Markov-Chain Monte Carlo, greedy algorithms, gradient descent, simulated annealing, etc. The list goes on and on. The best optimization algorithms to use for variational quantum circuits is currently an area of great interest. 

Below, we'll show how to use optimization algorithms in `scipy.optimize` to minimize quantum circuit cost functions. First, you are given a function which prepares a two-qubit state parameterized by two angles.

In [ ]:
"""Get a parameterized circuit."""
def circuit(params):
    qreg = qis.QuantumRegister(2)
    creg = qis.ClassicalRegister(2)
    circ = qis.QuantumCircuit(qreg, creg)
    circ.h(qreg)
    circ.cx(qreg[0], qreg[1])
    circ.rx(params[0], qreg[0])
    circ.ry(params[1], qreg[1])
    circ.cz(qreg[0], qreg[1])
    circ.s(qreg[0])
    circ.t(qreg[1])
    return circ

You can visualize the circuit by running the cell below.

In [ ]:
"""Visualize the circuit."""
print(circuit([1, 2]))

Next, you are given a Hamiltonian.

In [ ]:
"""Hamiltonian cost to minimize."""
weights = (1.2, -0.2)
paulis = ("IZ", "ZX")

<font size=8 color="#009600">&#9998;</font> **Do this:** In the following cell, write a function called `obj` which inputs **one argument**, a list of two floats which specify the values for the parameters, and returns the cost.

In [ ]:
"""Define a cost/objective function."""
def obj(params):
    """Returns the cost for the given parameters."""
    ### Your code here
    circ = circuit(params)
    val = cost(circ, weights, paulis)
    print("Current cost:", val, end="\r")
    return val
    

Now, run the cell below to test your function.

In [ ]:
"""Test your function on this set of parameters."""
obj([0, 0])

The cell below shows a minimum example of how to use a built-in optimization method from `scipy.optimize` to minimize the cost. Execute this cell to see what the minimum cost and best parameters are.

In [ ]:
"""Run an optimization algorithm to return the lowest cost and best parameters."""
result = minimize(obj, x0=[0, 0], method="COBYLA")

In [ ]:
"""See the optimization results."""
print("Lowest cost function value found:", result.fun)
print("Best parameters:", result.x)

You can try [other optimization methods in `scipy.optimize`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html), or you could write your own! And now that you know all the pieces of variational quantum algorithms, you can code your own up from scratch.

#### Congrats! You just implemented circuits with variational parameters, computed expectation values of Hamiltonians, and minimized objective functions. These are the ingredients necessary for any variational algorithm, including the QAOA.

# Further Reading and Resources

* [Theory of variational quantum algorithms](https://arxiv.org/abs/1509.04279)
* [Barren plateuas](https://arxiv.org/abs/1803.11173)
* [Stochastic gradient descent for variational circuits](https://arxiv.org/abs/1910.01155)